## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(400)

                    result_status = retrieve_ars_results(kcresult)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=891daf99-0a66-430d-ac53-c447c8809e60
Done
Done
ara-arax Done 259
Done
ara-aragorn Done 209
Done
ara-bte Done 205
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=70c22cec-e859-49dd-8418-ebf99f7fdaf7
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=6cc65509-eac0-48bf-85f6-e162953cb96b
Done
Done
ara-aragorn Done 438
Done
ara-arax Done 362
Done
ara-bte Done 83
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.Multiomics_BigGIM_DR_KP_RHOBTB2.json
A.Multiomics_BigGIM_DR_KP_RHOBTB2
https://arax.ncats.io/?source=ARS&id=ce3b655e-39fe-4926-bfb6-35d2b49d6c34
Done
Done
ara-aragorn

ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin.json
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
https://arax.ncats.io/?source=ARS&id=9c2188cb-371b-438a-b650-1b6187267cdf
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 500
Done
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin.json
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
https://arax.ncats.io/?source=ARS&id=87ec2cb7-9771-45e1-9986-13340fa897dc
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.1_parkinsons-crohns.json
D.1_parkinsons-crohns
https://arax.ncats.io/?source=ARS&id=1bdfedb2-5f54-4170-91bd-3d1b899f4e12
Done
Done
ara-aragorn Done 4399
Done
ara-arax ARS Error 0
Done
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minih

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [5]:
dict_workflows

{'A.3_KCNMA1': '891daf99-0a66-430d-ac53-c447c8809e60',
 'A.2a_RHOBTB2_twohop': '70c22cec-e859-49dd-8418-ebf99f7fdaf7',
 'A.8_EGFR_simple': '6cc65509-eac0-48bf-85f6-e162953cb96b',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': 'ce3b655e-39fe-4926-bfb6-35d2b49d6c34',
 'A.0_RHOBTB2_direct': '1c532958-563b-4de0-932e-1a8819596bfc',
 'A.2_RHOBTB2_twohop': '71a9f527-f2ea-48fd-b2dd-4a077d3db23e',
 'A.Multiomics_BigGIM_DR_KP_EGFR': '2a0c6ece-8e3c-4413-a43a-7b795c17fbeb',
 'A.2_RHOBTB2_twohop_constrained': '76894415-6916-4f84-aeae-6a628356641a',
 'A.9_EGFR_advanced': 'd58c612f-a015-4042-b777-f521d385407e',
 'A.2a_expanded_RHOBTB2_twohop_constrained': 'e9801a18-bf8e-4b2e-a4db-6e62268e6b14',
 'A.1_RHOBTB2': '0af496de-3c7e-42e7-86be-2890ab72b03a',
 'A.2a_expanded_RHOBTB2_twohop': '36ec6b6b-6181-402b-b848-2e5470b0b7cd',
 'B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': 'cc7fbaaf-917a-4201-8ed7-925257b59566',
 'B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone': 'e737dd83-dd3b-4671-81b9-54d3

In [6]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 891daf99-0a66-430d-ac53-c447c8809e60
Done
Done
ara-arax Done 259
Done
ara-aragorn Done 209
Done
ara-bte Done 205
A.2a_RHOBTB2_twohop 70c22cec-e859-49dd-8418-ebf99f7fdaf7
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
A.8_EGFR_simple 6cc65509-eac0-48bf-85f6-e162953cb96b
Done
Done
ara-aragorn Done 438
Done
ara-arax Done 362
Done
ara-bte Done 83
A.Multiomics_BigGIM_DR_KP_RHOBTB2 ce3b655e-39fe-4926-bfb6-35d2b49d6c34
Done
Done
ara-aragorn Done 3
Done
ara-arax Done 1
Done
ara-bte Done 4
A.0_RHOBTB2_direct 1c532958-563b-4de0-932e-1a8819596bfc
Done
Done
ara-arax Done 0
Done
ara-aragorn Done 0
Done
ara-bte Done 0
A.2_RHOBTB2_twohop 71a9f527-f2ea-48fd-b2dd-4a077d3db23e
Done
Done
ara-aragorn Done 8368
Done
ara-arax Done 500
Done
ara-bte Done 2231
A.Multiomics_BigGIM_DR_KP_EGFR 2a0c6ece-8e3c-4413-a43a-7b795c17fbeb
Done
Done
ara-aragorn Done 85
Done
ara-arax Done 20
Done
ara-bte Done 37
A.2_RHOBTB2_twohop_constrained 76894415-6916-4f84-aeae-6a628356641a
Done
Done
a

### Creating dataframe for workflows with PK

<br>

In [7]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [8]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [9]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=1c532958-...,https://arax.ncats.io/?source=ARS&id=0af496de-...,https://arax.ncats.io/?source=ARS&id=71a9f527-...,https://arax.ncats.io/?source=ARS&id=76894415-...,https://arax.ncats.io/?source=ARS&id=70c22cec-...,https://arax.ncats.io/?source=ARS&id=36ec6b6b-...,https://arax.ncats.io/?source=ARS&id=e9801a18-...,https://arax.ncats.io/?source=ARS&id=891daf99-...,https://arax.ncats.io/?source=ARS&id=6cc65509-...,https://arax.ncats.io/?source=ARS&id=d58c612f-...,...,https://arax.ncats.io/?source=ARS&id=af73b293-...,https://arax.ncats.io/?source=ARS&id=3e06ebad-...,https://arax.ncats.io/?source=ARS&id=2d2166b6-...,https://arax.ncats.io/?source=ARS&id=09852d0c-...,https://arax.ncats.io/?source=ARS&id=e57c4232-...,https://arax.ncats.io/?source=ARS&id=1bdfedb2-...,https://arax.ncats.io/?source=ARS&id=9ec02109-...,https://arax.ncats.io/?source=ARS&id=ef3728c7-...,https://arax.ncats.io/?source=ARS&id=d695e08a-...,https://arax.ncats.io/?source=ARS&id=c197d1a6-...
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200
ara-aragorn,No Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


### Creating second table with edge attribute source

<br>

In [10]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [11]:
final_dict2

defaultdict(dict,
            {'A.1_RHOBTB2': {'ara-arax': {'infores:arax', 'infores:rtx-kg2'},
              'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores:hetio',
               'infores:lincs',
               'infores:spoke'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'}},
             'A.2_RHOBTB2_twohop': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-covidkop',
               'infores:automat-ctd',
               'infores:automat-gtopdb',
               'infores:automat-hetio',
               'infores:automat-hmdb',
               'infores:automat-pharos',
               'infores:automat-robokop',
               'infores:bgee',
               'infores:connections-hypothesis',
               'infores:cord19-scibit

In [12]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [13]:
final_dictassemble

[['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:automat-robokop'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:hetio'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.2_RHOBTB2_twohop', 'ara-aragorn', 'infores:sider'],
 ['A.2_RHOBTB2_twohop', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.2_RHOBTB2_twohop', 'ara-aragorn', 'infores:hetio'],
 ['A.2_RHOBTB2_twohop', 'ara-aragorn', 'infores:sri-reference-kg'],
 ['A.2_RHOBTB2_twohop', 'ara-aragorn', 'infores:automat-covidkop'],
 ['A.2_RHOBTB2_twohop', 'ara-aragorn', 'infores:pharos'],
 ['A.2_RHOBTB2_twoh

In [14]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [15]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [16]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [17]:
df2test = pd.DataFrame(df2test.unstack().T)

In [18]:
df2test

Workflow,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,...,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN
JensenLab Knowledge UniProtKB-KW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN
JensenLab Text Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN
Monarch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN
UniProt Disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN
infores:aragorn,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],...,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:aragorn-ranker-ara,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],...,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]


In [19]:
#df2test.drop([''], axis=0, inplace=True)

In [20]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [21]:
#df2.replace([], 'None', regex=True,inplace=True)

In [22]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [23]:
df2test = df2test.rename_axis(None)

In [24]:
df2test.columns.name = None

In [25]:
df2test.fillna('', inplace=True)

In [26]:
df2test

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,...,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
CTD,,,,,,,,,,,...,,,,,,,,[ara-aragorn],[ara-aragorn],
DisGeNET,,,,,,,,,,,...,,,,,,,,[ara-aragorn],[ara-aragorn],
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,,,[ara-aragorn],,
JensenLab Knowledge UniProtKB-KW,,,,,,,,,,,...,,,,,,,,[ara-aragorn],,
JensenLab Text Mining,,,,,,,,,,,...,,,,,,,,[ara-aragorn],[ara-aragorn],
Monarch,,,,,,,,,,,...,,,,,,,,[ara-aragorn],,
UniProt Disease,,,,,,,,,,,...,,,,,,,,[ara-aragorn],,
infores:aragorn,[ara-aragorn],[ara-aragorn],,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],...,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:aragorn-ranker-ara,[ara-aragorn],[ara-aragorn],,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],...,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax]


In [27]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [28]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:335]
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

        
        
        
        
        
        
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]
df2test['Query Type'] = 'Async'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

In [29]:
df2test

,Query Type,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
infores:spoke,Async,KP,[ara-aragorn],[ara-aragorn],,[ara-aragorn],,,,,...,[ara-bte],,,[ara-bte],,,,[ara-aragorn],[ara-aragorn],
infores:molepro,Async,KP,,,,,,"[ara-arax, ara-bte]",,,...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],,,[ara-arax],[ara-arax],
infores:text-mining-provider-cooccurrence,Async,KP,,,,,,,,,...,[ara-bte],,,[ara-bte],,,,,,
infores:mychem-info,Async,KP,,,[ara-arax],,,,,,...,,,,,,,,,,
infores:mydisease-info,Async,KP,,,,,,,,,...,[ara-bte],,,[ara-bte],,,,,,
infores:connections-hypothesis,Async,KP,,"[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]",,,"[ara-arax, ara-aragorn, ara-bte]",,,...,,,,,,"[ara-aragorn, ara-arax]",,,,
infores:cohd,Async,KP,,,,,,,,,...,,,,,[ara-arax],"[ara-aragorn, ara-arax]",,[ara-arax],,"[ara-aragorn, ara-arax]"
infores:rtx-kg2,Async,KP,[ara-arax],"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,"[ara-aragorn, ara-arax]",[ara-arax],[ara-arax],"[ara-aragorn, ara-arax]",[ara-arax],,"[ara-arax, ara-aragorn]","[ara-aragorn, ara-arax]",[ara-arax],"[ara-aragorn, ara-arax]"
infores:biothings-semmeddb-gene,Async,KP,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,...,,,,,,,,,,
infores:biothings-multiomics-clinical-risk,Async,KP,,,,,,,,,...,,,,,,,,,,


<br>

## Converting the Pk's to hyperlink

<br>

In [30]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [31]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [32]:
df.sort_index(inplace=True)

In [33]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [34]:
df

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Async,No Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200
ara-arax,Async,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200
ara-bte,Async,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
pk,Async,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...",...,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."


In [35]:
#date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
#wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [36]:
#wks_name

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [37]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [38]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [39]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_09_27-08_32_27_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

existing.loc['pk'][:] = existing.loc['pk'][:].apply(lambda x: x.replace('" "', '","'))

existing.loc['pk'][1]

updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True)

In [40]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [41]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_09_27-08_32_27_AM")

In [42]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 0,
       'endRowIndex': 1,
       'startColumnIndex': 0,
       'endColumnIndex': 1}],
     'booleanRule': {'condition': {'type': 'NOT_BLANK'},
      'format': {'backgroundColor': {'red': 0.7176471,
        'green': 0.88235295,
        'blue': 0.8039216},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7176471,
         'green': 0.88235295,
         'blue': 0.8039216}}}}}}},
  {},
  {}]}

In [43]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 0,
       'endRowIndex': 1,
       'startColumnIndex': 0,
       'endColumnIndex': 1}],
     'booleanRule': {'condition': {'type': 'NOT_BLANK'},
      'format': {'backgroundColor': {'red': 0.7176471,
        'green': 0.88235295,
        'blue': 0.8039216},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7176471,
         'green': 0.88235295,
         'blue'

In [44]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red

In [45]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold

In [46]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 431338910,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
      

In [47]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_27-08_32_27_AM' id:431338910>

In [48]:
# Select a range
cell_list = sh.range('B19:AS19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_27-08_32_27_AM'!B19:AS19",
 'updatedRows': 1,
 'updatedColumns': 44,
 'updatedCells': 44}

In [49]:
# Select a range
cell_list = sh.range('B23:AS23')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_27-08_32_27_AM'!B23:AS23",
 'updatedRows': 1,
 'updatedColumns': 44,
 'updatedCells': 44}

In [50]:
set_frozen(sh, cols=1, rows=1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

### Push Dataframe 2

<br>

In [51]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_09_27-08_32_27_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws2.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

updated = existing.append(df2test)
gd.set_with_dataframe(ws2, updated,include_index=True,include_column_header=True)


In [52]:
set_frozen(ws2, cols=1, rows =1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [ ]:
sh